# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [48]:
# Do all imports and installs here
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
epoch = datetime.datetime(1960, 1, 1)
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf, col, count, when, isnan
import pyspark.sql.types as T

import io
import os
import configparser
import boto3

config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS_AdminFull']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_AdminFull']['AWS_SECRET_ACCESS_KEY']

In [49]:
spark = SparkSession \
        .builder \
        .config("spark.jars.repositories", "https://repos.spark-packages.org/") \
        .config("spark.jars.packages", "saurfang:spark-sas7bdat:3.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.2") \
        .getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

#### Immigration Data

In [50]:
df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [51]:
fname = 'immigration_port.csv'
df_immigration_port = pd.read_csv(fname)
df_immigration_port.dtypes

immigration_code    object
City                object
State_code          object
country             object
dtype: object

#### Airport Data

In [52]:
fname = 'airport-codes_csv.csv'
df_airport = pd.read_csv(fname)
df_airport.dtypes

ident            object
type             object
name             object
elevation_ft    float64
continent        object
iso_country      object
iso_region       object
municipality     object
gps_code         object
iata_code        object
local_code       object
coordinates      object
dtype: object

#### Weather Data

In [53]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_weather = pd.read_csv(fname)
df_weather.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

#### Demographics Data

In [54]:
fname = 'us-cities-demographics.csv'
df_cities = pd.read_csv(fname,sep=';')
df_cities.dtypes

City                       object
State                      object
Median Age                float64
Male Population           float64
Female Population         float64
Total Population            int64
Number of Veterans        float64
Foreign-born              float64
Average Household Size    float64
State Code                 object
Race                       object
Count                       int64
dtype: object

### Step 2: Explore and Assess the Data
### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.


#### Immigration Data

In [4]:
#List all parquet files in the repository

print(os.listdir('../../data/18-83510-I94-Data-2016/'))

['i94_apr16_sub.sas7bdat', 'i94_sep16_sub.sas7bdat', 'i94_nov16_sub.sas7bdat', 'i94_mar16_sub.sas7bdat', 'i94_jun16_sub.sas7bdat', 'i94_aug16_sub.sas7bdat', 'i94_may16_sub.sas7bdat', 'i94_jan16_sub.sas7bdat', 'i94_oct16_sub.sas7bdat', 'i94_jul16_sub.sas7bdat', 'i94_feb16_sub.sas7bdat', 'i94_dec16_sub.sas7bdat']


In [15]:
#Count rows
df_spark.count()

3096313

In [13]:
#List missing values
df_spark.select([count(when(isnan(c), c)).alias(c) for c in df_spark.columns]).show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender|insnum|airline|admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+
|    0|    0|     0|     0|     0|      0|      0|      0|      0|      0|     0|      0|    0|       0|       0|    0|      0|      0|      0|      0|      0|      0|     0|     0|      0|     0|    0|       0|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-

In [14]:
#List null values
df_spark.select([count(when(col(c).isNull(), c)).alias(c) for c in df_spark.columns]).show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|  occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender| insnum|airline|admnum|fltno|visatype|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+-------+-------+-------+------+-------+-------+------+-----+--------+
|    0|    0|     0|     0|     0|      0|      0|    239| 152592| 142457|   802|      0|    0|       1| 1881250|3088187|    238| 138429|3095921| 138429|    802|    477|414269|2982605|  83627|     0|19549|       0|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-------+-------+-------+---

##### Weather Data

In [16]:
#List columns with missing values
df_weather.loc[:, df_weather.isnull().any()].columns

Index(['AverageTemperature', 'AverageTemperatureUncertainty'], dtype='object')

In [19]:
#Count of all null values in the Dataframe
df_weather.isnull().sum().sum()

728260

In [22]:
df_weather.count()

dt                               8599212
AverageTemperature               8235082
AverageTemperatureUncertainty    8235082
City                             8599212
Country                          8599212
Latitude                         8599212
Longitude                        8599212
dtype: int64

#### Demographics

In [23]:
#List columns with missing values
df_cities.loc[:, df_cities.isnull().any()].columns

Index(['Male Population', 'Female Population', 'Number of Veterans',
       'Foreign-born', 'Average Household Size'],
      dtype='object')

In [25]:
#Count of all null values in the Dataframe
df_cities.isnull().sum().sum()

48

In [26]:
df_cities.count()

City                      2891
State                     2891
Median Age                2891
Male Population           2888
Female Population         2888
Total Population          2891
Number of Veterans        2878
Foreign-born              2878
Average Household Size    2875
State Code                2891
Race                      2891
Count                     2891
dtype: int64

##### Airports

In [29]:
#List columns with missing values
df_airport.loc[:, df_airport.isnull().any()].columns

Index(['elevation_ft', 'continent', 'iso_country', 'municipality', 'gps_code',
       'iata_code', 'local_code'],
      dtype='object')

In [30]:
#Count of all null values in the Dataframe
df_airport.isnull().sum().sum()

126968

In [31]:
df_airport.count()

ident           55075
type            55075
name            55075
elevation_ft    48069
continent       27356
iso_country     54828
iso_region      55075
municipality    49399
gps_code        41030
iata_code        9189
local_code      28686
coordinates     55075
dtype: int64


### Cleaning Steps
Document steps necessary to clean the data

#### Airports

- Re-arrange order of columns (remove iso_region)
- Filter on US only airports
- Delete duplicates

In [35]:
# Performing cleaning tasks here

df_us = df_airport.loc[(df_airport['iso_country'] == "US")]
df_us_airports = df_us[["iso_region","name","municipality","local_code","type","coordinates"]]
df_us_airports['state_code'] = df_us_airports['iso_region'].str[3:5]
df_us_airports["cityState"] = df_us_airports["state_code"]+df_us_airports["municipality"]
df_us_airports['cityState'] = df_us_airports['cityState'].str.lower()
df_us_airports.rename({'municipality': 'city'}, axis=1, inplace=True)
df_us_airports = df_us_airports[["cityState","state_code","name","city","local_code","type","coordinates"]]
df_us_airports.drop_duplicates()

df_us_airports.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

,cityState,state_code,name,city,local_code,type,coordinates
0,pabensalem,PA,Total Rf Heliport,Bensalem,00A,heliport,"-74.93360137939453, 40.07080078125"
1,ksleoti,KS,Aero B Ranch Airport,Leoti,00AA,small_airport,"-101.473911, 38.704022"
2,akanchor point,AK,Lowell Field,Anchor Point,00AK,small_airport,"-151.695999146, 59.94919968"
3,alharvest,AL,Epps Airpark,Harvest,00AL,small_airport,"-86.77030181884766, 34.86479949951172"
4,arnewport,AR,Newport Hospital & Clinic Heliport,Newport,NaN,closed,"-91.254898, 35.6087"


#### Cities

- Convert floats to int
- Split out into Cities / Races / Population
- Create new key "cityState"
- Remove duplicates

In [43]:
df_cities = df_cities.fillna(0)

df_cities['Male Population'] = df_cities['Male Population'].astype(int)
df_cities['Female Population'] = df_cities['Female Population'].astype(int)
df_cities['Number of Veterans'] = df_cities['Number of Veterans'].astype(int)
df_cities['Foreign-born'] = df_cities['Foreign-born'].astype(int)


df_cities['cityState'] = df_cities['State Code']+df_cities['City']
df_cities['cityState'] = df_cities['cityState'].str.lower()
df_races =  df_cities[['cityState','Race','Count']]

df_population = df_cities[['cityState','Median Age','Male Population','Female Population','Total Population','Number of Veterans','Foreign-born','Average Household Size']]
df_population = df_population.drop_duplicates()

df_cities = df_cities[['cityState','State Code','City','State']]
df_cities = df_cities.drop_duplicates()


In [44]:
df_cities.head(5)

,cityState,State Code,City,State
0,mdsilver spring,MD,Silver Spring,Maryland
1,maquincy,MA,Quincy,Massachusetts
2,alhoover,AL,Hoover,Alabama
3,carancho cucamonga,CA,Rancho Cucamonga,California
4,njnewark,NJ,Newark,New Jersey


In [45]:
df_population.head(5)

,cityState,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size
0,mdsilver spring,33.8,40601,41862,82463,1562,30908,2.60
1,maquincy,41.0,44129,49500,93629,4147,32935,2.39
2,alhoover,38.5,38040,46799,84839,4819,8229,2.58
3,carancho cucamonga,34.5,88127,87105,175232,5821,33878,3.18
4,njnewark,34.6,138040,143873,281913,5829,86253,2.73


In [47]:
df_races.head(5)

,cityState,Race,Count
0,mdsilver spring,Hispanic or Latino,25924
1,maquincy,White,58723
2,alhoover,Asian,4759
3,carancho cucamonga,Black or African-American,24437
4,njnewark,White,76402


#### Immigration

- Create new key cityState
- Filter on needed columns
- Convert SAS time to datetime
- Convert floats to integer where needed
- Reorder columns' names and order 

In [54]:
fname = 'immigration_port.csv'
df_immigration_port = pd.read_csv(fname)
df_immigration_port['cityState'] = df_immigration_port['State_code']+df_immigration_port['City']
df_immigration_port['cityState'] = df_immigration_port['cityState'].str.lower()
df_immigration_port['City'] = df_immigration_port['City'].str.title()
df_immigration_port = df_immigration_port[['cityState','City','State_code','country','immigration_code']]



immigration_table = df_spark.select(["i94port","i94yr","i94mon","arrdate","depdate","i94mode","i94addr","biryear","gender","occup","airline","fltno","entdepa","entdepd","entdepu","matflag","visatype"])

immigration_table = immigration_table.withColumnRenamed("i94port","entry_port") \
                        .withColumnRenamed("i94yr","year") \
                        .withColumnRenamed("i94mon","month") \
                        .withColumnRenamed("i94mode", "mode") \
                        .withColumnRenamed("biryear", "birthyear") \
                        .withColumnRenamed("occup", "occupation") \
                        .withColumnRenamed("fltno", "flightno") \
                        .withColumnRenamed("i94addr", "addr")

def convert_sas(sastime):
    try:
        return (pd.to_timedelta(sastime,unit='D')+ epoch)
    except:
        return None

convert_date = udf(lambda x: convert_sas(x),T.DateType())

immigration_table = immigration_table.withColumn('arrdate',convert_date(immigration_table.arrdate))\
                                    .withColumn('depdate',convert_date(immigration_table.depdate))

immigration_table = immigration_table.withColumn('year',col('year').cast('integer'))\
                .withColumn('month',col('month').cast('integer'))\
                .withColumn('birthyear',col('birthyear').cast('integer'))

In [46]:
df_immigration_port.head(5)

,cityState,City,State_code,country,immigration_code
0,akalcan,Alcan,AK,US,ALC
1,akanchorage,Anchorage,AK,US,ANC
2,akbaker aaf - baker island,Baker Aaf - Baker Island,AK,US,BAR
3,akdaltons cache,Daltons Cache,AK,US,DAC
4,akdew station pt lay dew,Dew Station Pt Lay Dew,AK,US,PIZ


In [48]:
immigration_table_pd = immigration_table.limit(10).toPandas()
immigration_table_pd

,entry_port,year,month,arrdate,depdate,mode,addr,birthyear,gender,occupation,airline,flightno,entdepa,entdepd,entdepu,matflag,visatype
0,XXX,2016,5,2016-05-31,None,NaN,None,1989,None,None,None,None,T,None,U,None,F1
1,XXX,2016,5,2016-05-24,None,NaN,None,1989,None,None,None,None,T,None,U,None,E2
2,XXX,2016,5,2016-05-04,None,NaN,None,1938,None,None,None,None,T,None,U,None,B2
3,XXX,2016,5,2016-05-27,None,NaN,None,1987,None,None,None,None,T,None,U,None,F1
4,CHI,2016,5,2016-05-03,2015-07-01,1.0,IL,1987,F,None,EK,00235,T,O,None,M,F1
5,XXX,2016,5,2016-05-31,None,0.0,None,1968,None,None,None,None,None,None,None,None,M1
6,BOS,2016,5,2016-05-01,2016-05-13,1.0,NY,1963,F,None,BA,215,T,O,None,M,B2
7,WAS,2016,5,2016-05-01,2016-05-10,1.0,NY,1950,None,None,OS,00093,O,O,None,M,B1
8,NYC,2016,5,2016-05-01,2016-05-30,1.0,NY,1948,None,None,AZ,00608,O,O,None,M,B2
9,MIA,2016,5,2016-05-01,2016-05-22,1.0,FL,1984,F,None,AZ,630,T,O,None,M,B2


#### Weather

- removing null temeprature rows
- filtering on US data
- Create year & month columns from date
- filter on recent data (from 2000)
- filter on needed columns

In [55]:
df_weather.dropna(subset = ["AverageTemperature"], inplace=True)

df_us = df_weather.loc[(df_weather['Country'] == "United States")]

df_us['dt'] = pd.to_datetime(df_us['dt'],format='%Y-%m-%d')

df_us['year'] = df_us['dt'].dt.year
df_us['month'] = df_us['dt'].dt.month

df_us = df_us.loc[(df_us['year'] > 1999)]

df_weather_us = df_us[['City','AverageTemperature','year','month']]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

In [56]:
df_weather_us.head(5)

,City,AverageTemperature,year,month
49715,Abilene,8.039,2000,1
49716,Abilene,11.908,2000,2
49717,Abilene,14.423,2000,3
49718,Abilene,18.274,2000,4
49719,Abilene,25.358,2000,5


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.